In [209]:
import pandas as pd
import numpy as np
import Levenshtein as lev
from fuzzywuzzy import process

# Read data
data = pd.read_excel('data.xlsx', sheet_name = 'PS19A_PERSON__SmplR10K')

# Shape data
data.shape

(999, 31)

In [210]:
Distance = lev.distance(data['person_address'][1].lower(),data['person_address'][2].lower())
print(Distance)
Ratio = lev.ratio(data['person_address'][1].lower(),data['person_address'][2].lower())
print(Ratio)
print(Ratio)

65
0.4
0.4


In [211]:
IndexRow= data['person_address']
ColumnRow= data['person_address']

FullFrame = pd.DataFrame(data, columns=IndexRow,index=ColumnRow)
FullFrame.head(5)
print(FullFrame['c/o Hitachi Appliances, Inc. 390 Muramatsu, Shimizu-ku,Shizuoka-shi, Shizuoka-ken 424-0926'][0])



nan


In [212]:
first_col  = FullFrame.iloc[:10,:10]
first_col.head(20)

person_address,"c/o Hitachi Appliances, Inc. 390 Muramatsu, Shimizu-ku,Shizuoka-shi, Shizuoka-ken 424-0926","NTT DoCoMo, Inc., Sanno Park Tower, 11-1, Nagata-,cho 2-chome, Chiyoda-ku, Tokyo 100-6150","Gas Chemical Company, Inc., 4-16-2, Hinagahigashi,Yokkaichi-shi MIE 510-0886","2-3, Marunouchi 2-chome, Chiyoda-ku,Tokyo 100-8310","Broomhaugh House Riding Mill,Northumberland NE44 6EG","1 Kimball Court, 504,Woburn, MA 01801","Kawasaki Steel Corporation 1, Kawasakicho Chuo-ku,Chiba-shi Chiba, 260-0835","Disco Corporation 13-11, Omori-Kita 2-chome Ota-ku,Tokyo 143-8580","1-201, Fukuzaki 3-chome, Minato-ku Osaka-shi,Osaka 552-0013","c/o Mitsui Chemicals, Inc. 1144, Togo,Mobara-shi, Chiba 297-0017"
person_address,,,,,,,,,,
"c/o Hitachi Appliances, Inc. 390 Muramatsu, Shimizu-ku,Shizuoka-shi, Shizuoka-ken 424-0926",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"NTT DoCoMo, Inc., Sanno Park Tower, 11-1, Nagata-,cho 2-chome, Chiyoda-ku, Tokyo 100-6150",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Gas Chemical Company, Inc., 4-16-2, Hinagahigashi,Yokkaichi-shi MIE 510-0886",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2-3, Marunouchi 2-chome, Chiyoda-ku,Tokyo 100-8310",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Broomhaugh House Riding Mill,Northumberland NE44 6EG",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1 Kimball Court, 504,Woburn, MA 01801",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Kawasaki Steel Corporation 1, Kawasakicho Chuo-ku,Chiba-shi Chiba, 260-0835",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Disco Corporation 13-11, Omori-Kita 2-chome Ota-ku,Tokyo 143-8580",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1-201, Fukuzaki 3-chome, Minato-ku Osaka-shi,Osaka 552-0013",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [214]:
FullFrame.head(10)

person_address,"c/o Hitachi Appliances, Inc. 390 Muramatsu, Shimizu-ku,Shizuoka-shi, Shizuoka-ken 424-0926","NTT DoCoMo, Inc., Sanno Park Tower, 11-1, Nagata-,cho 2-chome, Chiyoda-ku, Tokyo 100-6150","Gas Chemical Company, Inc., 4-16-2, Hinagahigashi,Yokkaichi-shi MIE 510-0886","2-3, Marunouchi 2-chome, Chiyoda-ku,Tokyo 100-8310","Broomhaugh House Riding Mill,Northumberland NE44 6EG","1 Kimball Court, 504,Woburn, MA 01801","Kawasaki Steel Corporation 1, Kawasakicho Chuo-ku,Chiba-shi Chiba, 260-0835","Disco Corporation 13-11, Omori-Kita 2-chome Ota-ku,Tokyo 143-8580","1-201, Fukuzaki 3-chome, Minato-ku Osaka-shi,Osaka 552-0013","c/o Mitsui Chemicals, Inc. 1144, Togo,Mobara-shi, Chiba 297-0017",...,"C/O MITSUBISHI ELECTRIC CORPORATION, 7-3, MARUNOUCHI 2-CHOME, CHIYODA-KU, TOKYO 1008310 〒1008310","C/O YAMADA BEE COMPANY, INC, 194 ICHIBA, KAGAMINO-CHO, TOMATA-GUN, OKAYAMA 7080393 〒7080393","156013, g. Kostroma, ul. Lenina, 19a, kv. 9, OOO 'Brend Agentstvo Zvezda'","440028, g. Penza, ul. Germana Titova, 28, FGBOU VO 'Penzenskij gosudarstvennyj universitet arkhitektury i stroitelstva'","C/O DENSO CORPORATION, 1-1, SHOWA-CHO, KARIYA-CITY AICHI 4488661 〒4488661","NO. 16 MEIHUA ROAD, ZHONGLOU ECONOMIC DEVELOPMENT DISTRICT CHANGZHOU, JIANGSU 213023","C/O ALPS ELECTRIC CO., LTD., 1-7, YUKIGAYA-OTSUKAMACHI, OTA-KU, TOKYO 1458501 〒1458501","C/O ZTE CORPORATION ZTE PLAZA, KEJI ROAD SOUTH, HI-TECH INDUSTRIAL PARK NANSHAN SHENZHEN, GUANGDONG 518057","129090, Moskva, ul. B. Spasskaja, 25, stroenie 3, OOO 'JUridicheskaja firma Gorodisskij i Partnery'","Standartizatsii i Sertifikatsii Veterinarnôh Preparatov, Zvenigorodskoje u. 5 , Moskva, RUSSIAN FEDERATION"
person_address,,,,,,,,,,,,,,,,,,,,,
"c/o Hitachi Appliances, Inc. 390 Muramatsu, Shimizu-ku,Shizuoka-shi, Shizuoka-ken 424-0926",1.000000,0.368715,0.445783,0.328571,0.309859,0.283465,0.412121,0.374194,0.402685,0.493506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"NTT DoCoMo, Inc., Sanno Park Tower, 11-1, Nagata-,cho 2-chome, Chiyoda-ku, Tokyo 100-6150",0.368715,1.000000,0.400000,0.575540,0.269504,0.301587,0.402439,0.558442,0.378378,0.392157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Gas Chemical Company, Inc., 4-16-2, Hinagahigashi,Yokkaichi-shi MIE 510-0886",0.445783,0.400000,1.000000,0.349206,0.265625,0.318584,0.476821,0.368794,0.370370,0.471429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2-3, Marunouchi 2-chome, Chiyoda-ku,Tokyo 100-8310",0.328571,0.575540,0.349206,1.000000,0.274510,0.344828,0.352000,0.539130,0.495413,0.350877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Broomhaugh House Riding Mill,Northumberland NE44 6EG",0.309859,0.269504,0.265625,0.274510,1.000000,0.314607,0.283465,0.290598,0.252252,0.310345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1 Kimball Court, 504,Woburn, MA 01801",0.283465,0.301587,0.318584,0.344828,0.314607,1.000000,0.339286,0.333333,0.354167,0.396040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Kawasaki Steel Corporation 1, Kawasakicho Chuo-ku,Chiba-shi Chiba, 260-0835",0.412121,0.402439,0.476821,0.352000,0.283465,0.339286,1.000000,0.471429,0.388060,0.446043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Disco Corporation 13-11, Omori-Kita 2-chome Ota-ku,Tokyo 143-8580",0.374194,0.558442,0.368794,0.539130,0.290598,0.333333,0.471429,1.000000,0.403226,0.387597,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1-201, Fukuzaki 3-chome, Minato-ku Osaka-shi,Osaka 552-0013",0.402685,0.378378,0.370370,0.495413,0.252252,0.354167,0.388060,0.403226,1.000000,0.455285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
import multiprocessing
import threading
import math

# Maybe use CPU count for number of threads?
print("CPU count: " + str(multiprocessing.cpu_count()));

cpu_count = multiprocessing.cpu_count()

# Number of threads to create
threadCount = 4;

first_col  = FullFrame.iloc[:10,:10]
first_col.head(20)


for row in first_col:
    for x in first_col.index:
        ratio = lev.ratio(x.lower(),row.lower())
        print(distance)
        FullFrame.at[row,x] = ratio
    

# Amount of addresses to use per thread
threadAddressCount = int(math.ceil(float(len(first_col)) / threadCount));

class calculateDistanceThread(threading.Thread):
    def __init__(self, threadID):
        threading.Thread.__init__(self)
        self.threadID = threadID
    def run(self):
        start = threadAddressCount * self.threadID
        for n in range(threadAddressCount):
            # Prevent index out of range in last thread
            if (start + n < len(addresses)):
                print("THREAD " + str(self.threadID) + ": " + addresses[start + n])
                # Calculate Levenshtein distance
                # and add to matrix

for n in range(threadCount):
    # Create thread with id n
    calculateDistanceThread(n).start()

CPU count: 8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\DePau\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\DePau\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\DePau\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\DePau\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
        self.run()
  File "<ipython-input-219-4a8479984915>", line 35, in run
    self.run()self.run()    
  File "<ipython-input-219-4a8479984915>", line 35, in run

self.run()
  File "<ipython-input-219-4a8479984915>", line 35, in run
  File "<ipython-input-219-4a8479984915>", line 35, in run
NameError: NameErrorname 'addresses' is not defined
NameError: name 'addresses' is not defined
: name 'addresses' i